In [24]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'vqa.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [25]:
df['cleaning_status'].value_counts()

cleaning_status
revised    242
Name: count, dtype: int64

In [26]:
df.head()

,cleaning_status,image_path,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_stratagy,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id,question
0,revised,val2014/COCO_val2014_000000304481.jpg,Is it cold outside? Answer with either 'yes' o...,Is it cold outside? Answer with either 'yes' o...,[yes],[],vqa,is it,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",304481,yes/no,304481002,Is it cold outside?
1,revised,val2014/COCO_val2014_000000219984.jpg,Is it cold outside? Answer with either 'yes' o...,Is it cold outside? Answer with either 'yes' o...,[no],[],vqa,is it,no,"[{'answer': 'no', 'answer_confidence': 'no', '...",219984,yes/no,219984008,Is it cold outside?
2,revised,val2014/COCO_val2014_000000566049.jpg,Is she throwing a frisbee? Answer with either ...,Is she throwing a frisbee? Answer with either ...,[yes],[],vqa,is,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",566049,yes/no,566049012,Is she throwing a frisbee?
3,revised,val2014/COCO_val2014_000000514598.jpg,Is she throwing a frisbee? Answer with either ...,Is she throwing a frisbee? Answer with either ...,[no],[],vqa,is,no,"[{'answer': 'no', 'answer_confidence': 'yes', ...",514598,yes/no,514598006,Is she throwing a frisbee?
4,revised,val2014/COCO_val2014_000000420330.jpg,Is there cheese in that dish? Answer with eith...,Is there cheese in that dish? Answer with eith...,[yes],[],vqa,is there,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",420330,yes/no,420330002,Is there cheese in that dish?


In [27]:
df['platinum_prompt'][1]

"Is it cold outside? Answer with either 'yes' or 'no'."

In [28]:
import pandas as pd
import re

# 1. Read the question again: ...
# def repeat_question(match):
#     # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
#     # match.group(2): 질문 텍스트
#     # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
#     # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
#     # 두 번째 줄바꿈을 그대로 붙인다.
#     return f"{match.group(1)}{match.group(2)}\n\nRead the question again: Think step by step. {match.group(2)}{match.group(3)}"

# # \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
# df['RE2'] = df['platinum_prompt'].str.replace(
#     r'(\n\s*\n)(.*?)(\n\s*\n)(?=Answer with either)',
#     repeat_question,
#     flags=re.DOTALL,
#     regex=True
# )

def transform_prompt(match):
    # group 1: 질문 부분 (첫 번째 물음표까지, 예: "Is it cold outside?")
    # group 2: 질문과 나머지 사이의 공백 (있을 경우)
    # group 3: 나머지 부분 (예: "Answer with either 'yes' or 'no'.")
    q = match.group(1)
    space = match.group(2)
    rest = match.group(3)
    return f"\"{q}{space}\n\nRead the question again: {q}\n\nThink step by step. {rest}\""

df['RE2'] = df['platinum_prompt'].str.replace(
    r'^(.*?\?)(\s*)(.*)$',
    transform_prompt,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question first. Think step by step. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a table first. Think step by step. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a graph structure first. Think step by step. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question in bullet points first. Think step by step. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Answer with either)',
    "Read the question again first. Think step by step. ",
    regex=True
)

In [29]:
df['RE2'][0]

'"Is it cold outside? \n\nRead the question again: Is it cold outside?\n\nThink step by step. Answer with either \'yes\' or \'no\'."'

In [21]:
df['platinum_prompt_no_cot'][0]

"Is it cold outside? Answer with either 'yes' or 'no'."

In [30]:
import pandas as pd
import re

# def repeat_question(match):
#     # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
#     # match.group(2): 질문 텍스트
#     # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
#     # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
#     # 두 번째 줄바꿈을 그대로 붙인다.
#     return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# # \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
# df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer in the format)',
#     repeat_question,
#     flags=re.DOTALL,
#     regex=True
# )

def transform_prompt(match):
    # group 1: 질문 부분 (첫 번째 물음표까지, 예: "Is it cold outside?")
    # group 2: 질문과 나머지 사이의 공백 (있을 경우)
    # group 3: 나머지 부분 (예: "Answer with either 'yes' or 'no'.")
    q = match.group(1)
    space = match.group(2)
    rest = match.group(3)
    return f"\"{q}{space}\n\nRead the question again: {q}\n\n{rest}\""

df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'^(.*?\?)(\s*)(.*)$',
    transform_prompt,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Answer with either)',
    "Read the question again first. ",
    regex=True
)

In [32]:
df['sRE2_no_cot'][0]

"Is it cold outside? Read the question again first. Answer with either 'yes' or 'no'."

In [33]:
df.to_parquet('vqa_add_task.parquet')

In [34]:
df.head()

,cleaning_status,image_path,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_stratagy,question_type,multiple_choice_answer,answers,...,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,revised,val2014/COCO_val2014_000000304481.jpg,Is it cold outside? Answer with either 'yes' o...,Is it cold outside? Answer with either 'yes' o...,[yes],[],vqa,is it,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",...,Is it cold outside? Organize the question into...,Is it cold outside? Organize the question into...,Is it cold outside? Summarize the question in ...,Is it cold outside? Read the question again fi...,"""Is it cold outside? \n\nRead the question aga...",Is it cold outside? Summarize the question fir...,Is it cold outside? Organize the question into...,Is it cold outside? Organize the question into...,Is it cold outside? Summarize the question in ...,Is it cold outside? Read the question again fi...
1,revised,val2014/COCO_val2014_000000219984.jpg,Is it cold outside? Answer with either 'yes' o...,Is it cold outside? Answer with either 'yes' o...,[no],[],vqa,is it,no,"[{'answer': 'no', 'answer_confidence': 'no', '...",...,Is it cold outside? Organize the question into...,Is it cold outside? Organize the question into...,Is it cold outside? Summarize the question in ...,Is it cold outside? Read the question again fi...,"""Is it cold outside? \n\nRead the question aga...",Is it cold outside? Summarize the question fir...,Is it cold outside? Organize the question into...,Is it cold outside? Organize the question into...,Is it cold outside? Summarize the question in ...,Is it cold outside? Read the question again fi...
2,revised,val2014/COCO_val2014_000000566049.jpg,Is she throwing a frisbee? Answer with either ...,Is she throwing a frisbee? Answer with either ...,[yes],[],vqa,is,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Summarize the quest...,Is she throwing a frisbee? Read the question a...,"""Is she throwing a frisbee? \n\nRead the quest...",Is she throwing a frisbee? Summarize the quest...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Summarize the quest...,Is she throwing a frisbee? Read the question a...
3,revised,val2014/COCO_val2014_000000514598.jpg,Is she throwing a frisbee? Answer with either ...,Is she throwing a frisbee? Answer with either ...,[no],[],vqa,is,no,"[{'answer': 'no', 'answer_confidence': 'yes', ...",...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Summarize the quest...,Is she throwing a frisbee? Read the question a...,"""Is she throwing a frisbee? \n\nRead the quest...",Is she throwing a frisbee? Summarize the quest...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Organize the questi...,Is she throwing a frisbee? Summarize the quest...,Is she throwing a frisbee? Read the question a...
4,revised,val2014/COCO_val2014_000000420330.jpg,Is there cheese in that dish? Answer with eith...,Is there cheese in that dish? Answer with eith...,[yes],[],vqa,is there,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",...,Is there cheese in that dish? Organize the que...,Is there cheese in that dish? Organize the que...,Is there cheese in that dish? Summarize the qu...,Is there cheese in that dish? Read the questio...,"""Is there cheese in that dish? \n\nRead the qu...",Is there cheese in that dish? Summarize the qu...,Is there cheese in that dish? Organize the que...,Is there cheese in that dish? Organize the que...,Is there cheese in that dish? Summarize the qu...,Is there cheese in that dish? Read the questio...
